In [1]:
%load_ext autoreload
%autoreload 2

# Testing: Run a single experiment

In [1]:
     import sys
     print(sys.executable)

/Users/leahtan/Desktop/2024-W/CPSC532Y/orthogonal_learning_codes/.venv/bin/python


In [1]:
from experiments import get_data_generator, exp, oracle_gen, dml, dml_split, dr, dr_split, myslearner

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


In [6]:
gen_data_fn, base_fn, tau_fn, prop_fn = get_data_generator('A', 1000, 6, .5)

In [3]:
method_list = [('dml_split', dml_split), ('slearner', myslearner)]

In [4]:
data = gen_data_fn()
results = exp(data, method_list, tau_fn, 4)

In [5]:
results

{'dml_split': 0.01726574846344717, 'slearner': 0.02008498088283078}

In [7]:
exp(data, [('dml', dml)], tau_fn, 4)

AutoMLWrap(model=<flaml.automl.model.LGBMEstimator object at 0x7fd3c8103940>)
AutoMLWrap(model=<flaml.automl.model.LGBMEstimator object at 0x7fd3c81a32e0>)


AttributeError: Cannot use a classifier as a first stage model when the target is continuous!

# Run all paper experiments

In [ ]:
import numpy as np
import os
import joblib
import matplotlib.pyplot as plt 
import pandas as pd
from experiments import main

### All CATE Experiments

In [ ]:
n_list = [500, 1000, 3000]
d_list = [6, 12]
sigma_list = [.5, 1, 2]
setup_list = ['A', 'B', 'C', 'D', 'E', 'F']
nx_list = [4, 4, 4, 5, 4, 4]
all_res = {}
for setup, n_x in zip(setup_list, nx_list):
    for n in n_list:
        for d in d_list:
            for sigma in sigma_list:
                target_dir = f'ortho_{setup}_{n}_{d}_{sigma}'
                print(target_dir)
                main(setup, n, d, n_x, sigma, 0, 2, target_dir)

In [ ]:
all_res = {}
for setup in setup_list:
    all_res[setup] = {}
    for n in n_list:
        all_res[setup][f'$n={n}$'] = {}
        for d in d_list:
            all_res[setup][f'$n={n}$'][f'$d={d}$'] = {}
            for sigma in sigma_list:
                all_res[setup][f'$n={n}$'][f'$d={d}$'][f'$\sigma={sigma}$'] = {}
                target_dir = f'ortho_{setup}_{n}_{d}_{sigma}'
                filename = os.path.join(target_dir, 'res.jbl')
                res = joblib.load(filename)
                results = {}
                for name in res[0].keys():
                    all_r = [r[name] for r in res]
                    results[f'{name}'] = f'{np.mean(all_r):.3f} ({np.std(all_r) / np.sqrt(len(all_r)):.3f})'

                all_res[setup][f'$n={n}$'][f'$d={d}$'][f'$\sigma={sigma}$'] = results
            all_res[setup][f'$n={n}$'][f'$d={d}$'] = pd.DataFrame(all_res[setup][f'$n={n}$'][f'$d={d}$']).T
        all_res[setup][f'$n={n}$'] = pd.concat(all_res[setup][f'$n={n}$'])
    all_res[setup] = pd.concat(all_res[setup])
    print(f'setup={setup}')
    display(all_res[setup])

### All Policy Experiments

In [ ]:
n_list = [500, 1000, 3000]
d_list = [6, 12]
sigma_list = [.5, 1, 2]

setup_list = ['A', 'B', 'D', 'E', 'F']
all_res = {}
for setup in setup_list:
    for n in n_list:
        for d in d_list:
            for sigma in sigma_list:
                target_dir = f'ortho_{setup}_{n}_{d}_{sigma}'
                print(target_dir)
                main(setup, n, d, d, sigma, 0, 2, target_dir, policy=True)

In [ ]:
all_res = {}
for setup in setup_list:
    all_res[setup] = {}
    for n in n_list:
        all_res[setup][f'$n={n}$'] = {}
        for d in d_list:
            all_res[setup][f'$n={n}$'][f'$d={d}$'] = {}
            for sigma in sigma_list:
                all_res[setup][f'$n={n}$'][f'$d={d}$'][f'$\sigma={sigma}$'] = {}
                target_dir = f'ortho_{setup}_{n}_{d}_{sigma}'
                filename = os.path.join(target_dir, 'res_policy.jbl')
                res = joblib.load(filename)
                results = {}
                for name in res[0].keys():
                    all_r = [r[name] for r in res]
                    results[f'{name}'] = f'{np.mean(all_r):.3f} ({np.std(all_r) / np.sqrt(len(all_r)):.3f})'

                all_res[setup][f'$n={n}$'][f'$d={d}$'][f'$\sigma={sigma}$'] = results
            all_res[setup][f'$n={n}$'][f'$d={d}$'] = pd.DataFrame(all_res[setup][f'$n={n}$'][f'$d={d}$']).T
        all_res[setup][f'$n={n}$'] = pd.concat(all_res[setup][f'$n={n}$'])
    all_res[setup] = pd.concat(all_res[setup])
    print(f'setup={setup}')
    display(all_res[setup])